In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Cargar los datos de phishing y no phishing
phishing_df = pd.read_excel('FISHING.xlsx')
non_phishing_df = pd.read_excel('NOFISHING.xlsx')

# Agregar una columna de etiqueta en ambos DataFrames
phishing_df['label'] = 1  # 1 indica phishing
non_phishing_df['label'] = 0  # 0 indica no phishing

# Combinar ambos DataFrames en uno solo
data = pd.concat([phishing_df, non_phishing_df], ignore_index=True)

# Guardar el DataFrame combinado en un nuevo archivo CSV
data.to_csv('combined_phishing_data.csv', index=False)  # 'index=False' evita que se guarde el índice

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer # Import TfidfVectorizer


# Cargar los datos combinados desde el archivo CSV
data = pd.read_csv('combined_phishing_data.csv')

# Revisar si hay valores nulos
print("Valores nulos:\n", data.isnull().sum())

# Rellenar valores nulos (si los hay) o eliminar filas
data.fillna(0, inplace=True)

# Separar las características (X) y las etiquetas (y)
X = data['url']  # Use the 'url' column as feature
y = data['label']  # Columna 'label'

# Create a TfidfVectorizer to convert URLs into numerical features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X) # Fit and transform the 'url' column


# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo de Bosque Aleatorio
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Evaluar el modelo
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))

# Guardar el modelo entrenado para uso futuro
joblib.dump(rf_model, 'random_forest_phishing_model.pkl')
print("Modelo guardado como 'random_forest_phishing_model.pkl'.")

Valores nulos:
 url      0
label    0
dtype: int64
Confusion Matrix:
 [[10588    19]
 [   17 18128]]

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     10607
           1       1.00      1.00      1.00     18145

    accuracy                           1.00     28752
   macro avg       1.00      1.00      1.00     28752
weighted avg       1.00      1.00      1.00     28752


Accuracy: 0.9987479131886478
Modelo guardado como 'random_forest_phishing_model.pkl'.


In [ ]:
from sklearn.metrics import accuracy_score

# Calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)

# Mostrar la exactitud en porcentaje
print(f"\nExactitud del modelo: {accuracy * 100:.2f}%")



Exactitud del modelo: 99.87%


In [ ]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Cargar el modelo entrenado
model_file = 'random_forest_phishing_model.pkl'
rf_model = joblib.load(model_file)
print("Modelo cargado correctamente.")

# Cargar el vectorizador utilizado durante el entrenamiento
vectorizer = TfidfVectorizer()
# Asegúrate de ajustar el vectorizador a los datos de entrenamiento originales
training_data = pd.read_csv('combined_phishing_data.csv')
vectorizer.fit(training_data['url'])

# Cargar los nuevos datos para pruebas
test_data_file = 'TESTING.xlsx'  # Cambia por la ruta de tu archivo
test_data = pd.read_excel(test_data_file)

# Verificar valores nulos
print("Valores nulos en los datos de prueba:\n", test_data.isnull().sum())
test_data.fillna('', inplace=True)  # Rellenar valores nulos con cadenas vacías

# Separar características (X) y etiquetas (y) en los datos de prueba
X_test = vectorizer.transform(test_data['url'])  # Convertir URLs a características numéricas
y_test = test_data['label']

# Realizar predicciones
y_pred = rf_model.predict(X_test)


print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nAccuracy:", accuracy_score(y_test, y_pred))


test_data['predicted_label'] = y_pred
test_data.to_csv('testing_results.csv', index=False)
print("Resultados guardados en 'testing_results.csv'.")


Modelo cargado correctamente.
Valores nulos en los datos de prueba:
 url      0
label    0
dtype: int64

Confusion Matrix:
 [[ 149 1739]
 [  14 1786]]

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.08      0.15      1888
           1       0.51      0.99      0.67      1800

    accuracy                           0.52      3688
   macro avg       0.71      0.54      0.41      3688
weighted avg       0.72      0.52      0.40      3688


Accuracy: 0.5246746203904555
Resultados guardados en 'testing_results.csv'.
